In [1]:
import pandas as pd

The goal of this notebook is to clean up the publication year field from the inventory of the library.
This will allow a more standarized field that is better fot filtering and analysis.

In [2]:
inventory = pd.read_csv('./seattle-library-checkout-records/inventory_unique.csv')
inventory.rename(columns = {'BibNum': 'BibNumber'}, inplace = True)


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### get all unique values and their count, to solve the more frequently occurent issues

In [3]:
unique_years = inventory.groupby('PublicationYear').size().sort_values(ascending = False)
unique_years = unique_years.reset_index().rename(columns = {0: 'cnt'})
unique_years.head()

,PublicationYear,cnt
0,[2015],21161
1,[2016],20432
2,[2014],19129
3,[2017],18499
4,2016.,18063


### clean whatever possible

In [29]:
def clean_pub_year(string):
    try: 
        #capture the cases that are of the form 2012, c2011
        if string[4]==',' and string[5]==' ' and (string[6]=='c' or string[6]=="©"):
            new_string = string[6:]
        else:
            new_string = string

        new_string = new_string.replace("[","").replace("]","").replace(".","").replace("""'""","")\
                   .replace("c","").replace('"','').replace("(","").replace(")","")\
                  .replace(",","").replace("-","").replace("?","")\
                  .replace("p","").replace("©",'').replace("℗","")
        if len(new_string)==4:
            return int(new_string)
        else:
            return string
    except:
        return string

In [30]:
unique_years['PublicationYear'] = unique_years['PublicationYear'].apply(lambda x: clean_pub_year(x))

### see how many values are covered

In [31]:
total = unique_years['cnt'].sum()
print(total)

not_covered = unique_years[unique_years['PublicationYear'].apply(lambda x: type(x)!=int)]['cnt'].sum()
print('covered', (total-not_covered)/total)

1122120
covered 0.9709897337183189


### see what values remain still problematic

In [32]:
not_resolved = unique_years[unique_years['PublicationYear'].apply(lambda x: type(x)!=int)].groupby('PublicationYear').sum().reset_index()
not_resolved.sort_values(by='cnt', ascending = False, inplace = True)

not_resolved.head(20)

,PublicationYear,cnt
11324,[date of publication not identified],809
10765,[between 1900-1999],560
5446,[19--],529
10836,[between 1980-1989?],465
7238,[198-?],401
10809,[between 1970-1979?],263
5443,[19--?],241
10762,[between 1900-1999?],240
10937,[between 1990-1999?],232
6734,[197-?],210


### when we are satisfied with the accuracy, then we can implement the change in the inventory file

In [33]:
inventory['PublicationYear'] = inventory['PublicationYear'].apply(lambda x: clean_pub_year(x))

In [35]:
inventory[inventory['PublicationYear'].apply(lambda x: type(x)==int)].to_csv('./seattle-library-checkout-records/inventory_unique_clean_pub_year.csv', index = False)

### vis

In [2]:
inventory = pd.read_csv('./seattle-library-checkout-records/inventory_unique_clean_pub_year.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
inventory.groupby('PublicationYear').size().sort_values(ascending = False)[:10]